In [ ]:
### Package Installation ###
pip install perceval-quandela

In [11]:
from IPython import display
from collections import Counter
from tabulate import tabulate
from tqdm.auto import tqdm
import gzip
import pickle
import time

import sympy as sp
import random

import perceval as pcvl
import perceval.lib.symb as symb

n = 14 # Number of photons at the input
m = 60 # Number of modes
N = 2  # Number of samples

Unitary_60 = pcvl.Matrix.random_unitary(m) # Creates a random unitary of dimension 60

# Define a 2-mode unitary circuit that we can use to decompose the 60 mode unitary
mzi = (symb.BS() // (0, symb.PS(phi=pcvl.Parameter("φ_a")))
       // symb.BS() // (1, symb.PS(phi=pcvl.Parameter("φ_b"))))
pcvl.pdisplay(mzi)

In [12]:
# Decompose the unitary into a Reck's type circuit
Linear_Circuit_60 = pcvl.Circuit.decomposition(Unitary_60, mzi,
                                               phase_shifter_fn=symb.PS,
                                               shape="triangle")

# Sampling using CliffordClifford2017 backend
Sampling_Backend = pcvl.BackendFactory().get_backend("CliffordClifford2017")

# Select a random input
def Generating_Input(n, m, modes = None):
    "This function randomly chooses an input with n photons in m modes."
    if modes == None :
        modes = sorted(random.sample(range(m),n))
    state = "|"
    for i in range(m):
        state = state + "0"*(1 - (i in modes)) +"1"*(i in modes)+ ","*(i < m-1)
    return pcvl.BasicState(state + ">")

# Display input state
input_state = Generating_Input(n, m)
print("The input state: ", input_state)

The input state:  |0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0>


In [33]:
bitMax = 10000; # We want to generate at least bitMax random bits
bitNum = 0; # Counter for bits generated

import time
start_time = time.time()

while bitNum <= bitMax:
    
    S1 = str(Sampling_Backend(Unitary_60).sample(pcvl.BasicState(input_state))) # Saves Sample 1
    S2 = str(Sampling_Backend(Unitary_60).sample(pcvl.BasicState(input_state))) # Saves Sample 2
    finalString = [] # Saves final string after von-Neuman post processing

    for x in range(2*m+1):
        if S1[x] == '|' or S1[x] == ',' or S1[x] =='>':
            pass
        else:
            # print(S1[x], S2[x])
            if (int(S1[x]) > 0 and int(S2[x]) > 0) or (int(S1[x]) == 0 and int(S2[x]) == 0):
                pass
            else:
                if int(S1[x]) > 0 and int(S2[x])==0:
                    finalString.append(0);
                else:
                    finalString.append(1);

    # Append the generated random number in ASCII to the end of a txt file
    # Will create file if it does not already exist in directory
    f = open('RandBits.txt', 'a')
    for x in range(len(finalString)):
        f.write(str(finalString[x]))
    f.close()
    
    bitNum += len(finalString)
    
print("%d bits / %s seconds" % (bitNum, time.time() - start_time))
print("= %d bits/sec" % (bitNum/(time.time() - start_time)))

10016 bits / 2.3230950832366943 seconds
= 4311 bits/sec
